## 

Autor: **Radosław Dąbkowski**

Numer indeksu: **325683**

Grupa: **1**
Numer: **2**

### Wstęp



In [158]:
# -*- coding: utf-8 -*-
"""
Created on Sat Dec  3 14:08:04 2022

@author: mgrzy
"""

import numpy as np
import gpxpy
import scipy
import gpxpy.gpx
import pandas as pd

def gpx2df(gpx_file_name, new_file_name = 'route_df.csv'):
    with open(gpx_file_name, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    route_info = []
    
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                    'time':point.time
                })
    
    route_df = pd.DataFrame(route_info)
    
    save_df(route_df, new_file_name)

def blh2xyz(fi, lam, h) -> np.array:
    '''
    Parameters
    ----------
    fi : float
        latitude [rad].
    lam : float
        longitude [rad].
    h : float
        height [m].
    Returns
    -------
    xyz : numpy array
        x, y, z [m].
    '''
    a = 6378137.0
    e2 = 0.00669438002290
    N = a/np.sqrt(1-e2*np.sin(fi)**2)
    x = (N+h)*np.cos(fi)*np.cos(lam)
    y = (N+h)*np.cos(fi)*np.sin(lam)
    z = (N*(1-e2)+h)*np.sin(fi)
    xyz = np.array([x, y, z])
    return xyz

def save_df(route_df, new_file_name= 'route_df.csv'):
    route_df.to_csv(new_file_name, index=False)
    print(f'file saved to {new_file_name}')
def read_route_df(file):
    route_df = pd.read_csv(file)
    return route_df

In [159]:
gpx2df('Afternoon_Ride.gpx')
df = read_route_df('route_df.csv')
df.head()

file saved to route_df.csv


,latitude,longitude,elevation,time
0,53.208030,23.326793,136.4,2022-07-28 13:26:13+00:00
1,53.208035,23.326818,136.4,2022-07-28 13:26:14+00:00
2,53.208038,23.326861,136.4,2022-07-28 13:26:16+00:00
3,53.208047,23.326925,136.3,2022-07-28 13:26:17+00:00
4,53.208052,23.326948,136.3,2022-07-28 13:26:18+00:00


In [160]:
import folium

m = folium.Map(location=[df.latitude.mean(), df.longitude.mean()], zoom_start=12)

l = list(zip(df.latitude.values, df.longitude.values))

folium.PolyLine(l, color="red", weight=2.5, opacity=1).add_to(m)

# add points that show the elevation

for i in range(0, len(df), 100):
    folium.Circle(
        location=[df.iloc[i]['latitude'], df.iloc[i]['longitude']],
        popup=df.iloc[i]['elevation'],
        radius=10,
        color='crimson',
        fill=True,
        fill_color='crimson'
    ).add_to(m)

phis = df['latitude'].values
lambdas = df['longitude'].values
dists = []
for i in range(len(phis)-1):
    phi1 = phis[i]
    lam1 = lambdas[i]
    phi2 = phis[i+1]
    lam2 = lambdas[i+1]

    xyz1 = blh2xyz(np.deg2rad(phi1), np.deg2rad(lam1), 0)
    xyz2 = blh2xyz(np.deg2rad(phi2), np.deg2rad(lam2), 0)
    dist = np.linalg.norm(xyz1-xyz2)
    dists.append(dist)

# append the last distance
phi1 = phis[-1]
lam1 = lambdas[-1]
phi2 = phis[0]
lam2 = lambdas[0]

xyz1 = blh2xyz(np.deg2rad(phi1), np.deg2rad(lam1), 0)
xyz2 = blh2xyz(np.deg2rad(phi2), np.deg2rad(lam2), 0)
dist = np.linalg.norm(xyz1-xyz2)
dists.append(dist)

df['distance'] = np.cumsum(dists)

# sample data
# 52.81 14.81      36.68
# 52.81 14.82      36.67
# 52.81 14.83      36.66
# 52.81 14.84      36.65
# 52.81 14.85      36.63
# 52.81 14.86      36.62
# 52.81 14.87      36.61
# 52.81 14.88      36.59
# 52.81 14.89      36.58
# 52.81 14.90      36.57
# 52.81 14.91      36.55
# 52.81 14.92      36.54
# 52.81 14.93      36.53
# 52.81 14.94      36.51
# 52.81 14.95      36.50
model = np.genfromtxt('siatka_gugik-geoid2011-PL-KRON86-NH.txt', skip_header=1)
model = model[~np.isnan(model[:, 2])]

# create a dict with keys: (phi, lam) and values: h
model_dict = {}
for row in model:
    model_dict[(row[0], row[1])] = row[2]

def interpolate_geoid(model, phi, lam, tr):
    '''
    Parameters
    ----------
    model : numpy array
        geoid model.
    phi : float
        latitude [deg].
    lam : float
        longitude [deg].
    Returns
    -------
    h : float
        geoid height [m].
    '''
    # find the 4 closest points
    # eg.
    # point: phi=52.812, lam=14.814
    # phi lam h
    # 52.81 14.81      36.68
    # 52.81 14.82      36.67
    # 52.82 14.81      36.66
    # 52.82 14.82      36.65
    # result is the weighted average height of the 4 closest points
    # h = (h1*d1 + h2*d2 + h3*d3 + h4*d4)/(d1+d2+d3+d4)
    # where d1, d2, d3, d4 are distances between the point and the closest points
    # h1, h2, h3, h4 are heights of the closest points
    
    # find the 4 closest points
    d, idx = tr.query([phi, lam], k=4)
    # calculate weights
    w = 1/d
    # calculate heights
    h = model[idx, 2]
    # calculate weighted average height
    h1 = np.sum(h*w)/np.sum(w)
    return h1

def multi_interpolate_geoid(model, phis, lams):
    '''
    Parameters
    ----------
    model : numpy array
        geoid model.
    phis : numpy array
        latitudes [deg].
    lams : numpy array
        longitudes [deg].
    Returns
    -------
    hs : numpy array
        geoid heights [m].
    '''
    # create a kd-tree
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html
    # https://en.wikipedia.org/wiki/K-d_tree
    # https://en.wikipedia.org/wiki/Nearest_neighbor_search
    hs = []
    tr = scipy.spatial.KDTree(model[:, :2])
    for phi, lam in zip(phis, lams):
        h1 = interpolate_geoid(model, phi, lam, tr)
        hs.append(h1)
    return hs
    

# interpolate geoid
for phi, lam in zip(phis, lambdas):
    h1 = interpolate_geoid(model, phi, lam)
    print(f'phi: {phi}, lam: {lam}, h1: {h1}')
# show model
model

phi: 53.20803, lam: 23.326793, h1: 27.97153893556515
phi: 53.208035, lam: 23.326818, h1: 27.971533216707286
phi: 53.208038, lam: 23.326861, h1: 27.971524272083496
phi: 53.208047, lam: 23.326925, h1: 27.971510049325662
phi: 53.208052, lam: 23.326948, h1: 27.97150459387994
phi: 53.208065, lam: 23.327002, h1: 27.97149147803627
phi: 53.208084, lam: 23.327062, h1: 27.971475962246018
phi: 53.208101, lam: 23.327113, h1: 27.971462491586774
phi: 53.208127, lam: 23.327186, h1: 27.97144269587502
phi: 53.208161, lam: 23.327289, h1: 27.971414783303686
phi: 53.208202, lam: 23.327412, h1: 27.971380667259627
phi: 53.208258, lam: 23.327584, h1: 27.971331860897138
phi: 53.208287, lam: 23.327668, h1: 27.97130712390287
phi: 53.208316, lam: 23.32775, h1: 27.97128244950187
phi: 53.208344, lam: 23.327832, h1: 27.971257575290586
phi: 53.208372, lam: 23.327914, h1: 27.971232302826746
phi: 53.208401, lam: 23.327995, h1: 27.971206661362416
phi: 53.208431, lam: 23.328076, h1: 27.97118039300754
phi: 53.20846, lam:

array([[49.  , 22.65, 37.41],
       [49.  , 22.66, 37.39],
       [49.  , 22.67, 37.37],
       ...,
       [54.88, 18.29, 29.36],
       [54.88, 18.3 , 29.34],
       [54.88, 18.31, 29.32]])